In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Local imports
from data_fetcher import DataFetcher
# from preprocessing import Processor

In [4]:
# Define some string constants for easy typing
SAMPLE_DATA_BY_SITE = 'sampleData/bySite'
SAMPLE_DATA_BY_COUNTY = 'sampleData/byCounty'
SAMPLE_DATA_BY_STATE = 'sampleData/byState'
SAMPLE_DATA_BY_BOX = 'sampleData/byBox'
SAMPLE_DATA_BY_CBSA = 'sampleData/byCBSA'

LIST_STATES = 'list/states'
LIST_COUNTIES_BY_STATE = 'list/countiesByState'
LIST_SITES_BY_COUNTY = 'list/sitesByCounty'
LIST_CBSAs = 'list/cbsas'
LIST_PARAM_CLASSES = 'list/classes'
LIST_PARAM_IN_CLASS = 'list/parametersByClass'

In [5]:
datafetcher = DataFetcher()

In [6]:
# Example calls to list codes
cali_code = datafetcher.get_codes(LIST_STATES, all=False, value='California')
print('California state code:', cali_code)

criteria_code = datafetcher.get_codes(LIST_PARAM_CLASSES, all=False, value='Criteria Pollutants')
print('Criteria polutants code:', criteria_code)

carbon_monoxide_code = datafetcher.get_codes(LIST_PARAM_IN_CLASS, all=False, value='Carbon monoxide', nparams={'pc':criteria_code})
print('Carbon monoxide code:', carbon_monoxide_code)

California state code: 06
Criteria polutants code: CRITERIA
Carbon monoxide code: 42101


In [7]:
datafetcher.all_codes

,value_represented
code,
11101,Suspended particulate (TSP)
11102,Suspended particulate (TSP) LC
11103,Benzene soluble organics (TSP)
11104,Total polynuclear hydrocarbons
11114,Windblown particulate
...,...
88500,PM2.5 Total Atmospheric
88501,PM2.5 Raw Data
88502,Acceptable PM2.5 AQI & Speciation Mass


In [8]:
_ = datafetcher.find_code('Cristabalite', verbose=True)

Cristabalite code is: 11122


## Explore data by area codes to find good location for modelling

We want to find a site, or small grroup of sites, that have enough data for us to train a useeful model. We need this set to contain metereological, ozone, particulate matter, and pollutant data.

We sample a day per year for 5 of the last 20 years and find the site with the best data for this particular county/state pair. We do this sampling because the API will lock us out if we try to get too much yearly data.

This code takes roughly 30 minutes to run. And it finds that Los Angels-North Main Street had the most data.

In [9]:
# r = datafetcher.find_best_location()
# with open('data.json', 'w') as fp:
#     json.dump(r, fp)

with open('data.json', 'r') as f:
  r = json.load(f)

Let's find which sites have the most data

In [10]:
data = r['Data']
metadata = r['Metadata']

num_codes = len(data['Azusa']) # Any key in result dict
num_years = len(data['Azusa'][0])

yearly_results = {site:[sum([data[site][code][year] for code in range(num_codes)]) for year in range(num_years)] for site in data}
yearly_best = {site: ([date for date, val in enumerate(yearly_results[site]) if val == max(yearly_results[site])], max(yearly_results[site])) for site in yearly_results}
yearly_best_sorted = sorted(yearly_best.items(), key=lambda x : x[1][1], reverse=True)
yearly_best_sorted[:5] # NOTE: (year whree most measurements weree takeen, most mausrements taken)

[('Burbank', ([2], 13)),
 ('Los Angeles-North Main Street', ([3, 4], 13)),
 ('Azusa', ([1], 12)),
 ('Pico Rivera #2', ([2], 12)),
 ('Santa Clarita', ([2], 12))]

In [11]:
dates = [i[0]+':'+i[1] for i in metadata['dates']]
codes = [datafetcher.all_codes.loc[code]['value_represented'] for code in metadata['codes']]

ndf = pd.DataFrame(data['Los Angeles-North Main Street'])
ndf.index = codes
ndf.columns = dates
ndf # NOTE: Table for when we have values in chosen station!

,20000528:20000529,20051030:20051031,20100301:20100302,20150413:20150414,20200702:20200703
Carbon monoxide,True,True,True,True,True
Nitrogen dioxide (NO2),True,True,True,True,True
Ozone,True,True,True,True,True
PM2.5 - Local Conditions,True,True,True,True,True
Wind Direction - Resultant,True,True,False,True,True
Wind Speed - Resultant,True,True,False,True,True
Outdoor Temperature,True,True,True,True,True
Relative Humidity,True,True,True,True,True
Solar radiation,True,False,False,True,True
Ultraviolet radiation,False,False,False,True,True


From the results above we will proceed with <b>Los Angeles-North Main Street</b> as our primary location to model. Notice that it has data on all our criteria pollutants and MET variables. We now proceed to find the amount of VOC data we have for these sites with the same sampling as before.

<b>SAVE CODES AND YEARS IN DICT TOO, USE DATES TO SEARCH FOR VOC</b>

In [12]:
# Pick 5 best sites
best_sites = [i[0] for i in yearly_best_sorted[:5]]
best_sites_codes = [datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value=i, nparams={'state':'06', 'county':'037'}) for i in best_sites]
best_sites_dates = [[metadata['dates'][j] for j in i[1][0]] for i in yearly_best_sorted[:5]]
best_sites

['Burbank',
 'Los Angeles-North Main Street',
 'Azusa',
 'Pico Rivera #2',
 'Santa Clarita']

In [13]:
# voc_r = datafetcher.find_voc_availability(best_sites, best_sites_codes, best_sites_dates)
# with open('voc_data.json', 'w') as f:
#     json.dump(voc_r, f)

with open('voc_data.json', 'r') as f:
  voc_r = json.load(f)

In [14]:
voc_data = np.array(voc_r['Data'])
voc_df = pd.DataFrame(voc_r['Data'])
voc_df.index = voc_r['Metadata']['codes']
voc_df.head(5)

,Burbank,Los Angeles-North Main Street,Azusa,Pico Rivera #2,Santa Clarita
43000,[False],"[False, True]",[True],[False],[False]
43102,[False],"[False, True]",[True],[False],[False]
43202,[False],"[False, True]",[True],[False],[False]
43203,[False],"[False, True]",[True],[False],[False]
43204,[False],"[False, True]",[True],[False],[False]


In [15]:
voc_site_results = {}
for site in voc_r['Data']:
    arr = np.array(voc_r['Data'][site])
    voc_site_results[site] = arr.sum(axis=0)
voc_site_results

{'Burbank': array([-1]),
 'Los Angeles-North Main Street': array([ 0, 59]),
 'Azusa': array([56]),
 'Pico Rivera #2': array([0]),
 'Santa Clarita': array([0])}

In [16]:
voc_r['Metadata']['dates']

[[['20100301', '20100302']],
 [['20150413', '20150414'], ['20200702', '20200703']],
 [['20051030', '20051031']],
 [['20100301', '20100302']],
 [['20100301', '20100302']]]

### Explore dataset for chosen sight W VOC data

Los Angeles-North Main Street had the most CRITERIA, MET, and VOC data (almost all the PAMS_VOCS are in this data set) <b>for the sampled date in 2020</b>

In [17]:
print(yearly_best_sorted[1])
print(best_sites_codes[1])

('Los Angeles-North Main Street', ([3, 4], 13))
1103


In [18]:
# df = datafetcher.create_dataset(20200101, 20200102, site='1103', county='037', state='06', processed=True, verbose=False, vocs=True) # NOTE: This crashes because my kernel on my computer can't handle the computation.
# df.shape

### Include Emissions projections for chosen sight W/O VOC data

In [19]:
site_code = datafetcher.get_codes(LIST_SITES_BY_COUNTY, all=False, value='Los Angeles-North Main Street', nparams={'state':'06', 'county':'037'})

# Example of Site data using Los Angeles-North Main Street, Los Angeles, California
df = datafetcher.create_dataset(20180101, 20181231, site=site_code, county='037', state='06', processed=True, verbose=False, vocs=False)

/Users/tcintra/Desktop/research/chem150/chem150/venv/lib/python3.7/site-packages/pandas/core/frame.py:9166: FutureWarning: Passing 'suffixes' which cause duplicate columns {'longitude_x', 'latitude_x'} in the result is deprecated and will raise a MergeError in a future version.
  joined, frame, how=how, left_index=True, right_index=True


In [20]:
# NOTE: The (lat, lon) for LA North Main St is (34.07, -118.23)
# NOTE: The HEMCO CEDS netcdf files use bounding box every 50 km, so we want the closest box (34.25, -118.25)
print(df.shape)
df

(8760, 13)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,Barometric pressure,latitude,longitude
datetime,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,1009.0,34.06659,-118.22688
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,1009.0,34.06659,-118.22688
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,999.0,34.06659,-118.22688
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,1000.0,34.06659,-118.22688
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,1001.0,34.06659,-118.22688


In [21]:
nc_links, _ = datafetcher.get_ceds_links(year='2018')
la_lat = 34.25
la_lon = -118.25
ceds_df = datafetcher.make_ceds_df(la_lat, la_lon, nc_links)
ceds_df

,ALD2_agr,ALD2_ene,ALD2_ind,ALD2_rco,ALD2_shp,ALD2_slv,ALD2_tra,ALD2_wst,ALK4_butanes_agr,ALK4_butanes_ene,...,TOLU_tra,TOLU_wst,XYLE_agr,XYLE_ene,XYLE_ind,XYLE_rco,XYLE_shp,XYLE_slv,XYLE_tra,XYLE_wst
2018-01-01 00:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 01:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 02:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 03:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-01-01 04:00:00,0.0,1.268880e-15,3.655395e-14,4.180411e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.349363e-12,...,1.650062e-12,3.429407e-13,0.0,2.333671e-13,1.001630e-11,1.231820e-13,1.085941e-14,4.498998e-11,1.357164e-12,2.165785e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 20:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 21:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13
2018-12-31 22:00:00,0.0,1.270339e-15,3.655395e-14,4.400093e-13,0.0,0.0,1.599644e-13,1.295176e-13,0.0,1.350914e-12,...,1.650062e-12,3.429407e-13,0.0,2.336354e-13,1.001630e-11,1.296552e-13,1.147636e-14,4.498998e-11,1.357164e-12,2.165785e-13


In [22]:
# Aggregate CEDS Emissions data
ceds_df_aggregated = datafetcher.aggregate_ceds_data(ceds_df)
print("Before Aggregating:", ceds_df.shape)
print("After Aggregating:", ceds_df_aggregated.shape)
ceds_df_aggregated.head(5)

Before Aggregating: (8760, 248)
After Aggregating: (8760, 31)


,ALD2,ALK4_butanes,ALK4_hexanes,ALK4_pentanes,BC,BENZ,BUTENE,C2H2,C2H4,C2H6,...,NH3,NO,OC,OTHER_AROM,OTHER_VOC,PRPE,SO2,TMB,TOLU,XYLE
2018-01-01 00:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 01:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 02:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 03:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 04:00:00,7.453460e-13,2.052799e-11,2.124349e-10,2.954461e-11,5.321832e-12,8.560323e-12,7.575214e-12,2.241559e-12,5.048348e-12,2.606391e-12,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11


In [23]:
big_df = df.join(ceds_df_aggregated, how='inner')
print(big_df.shape)
big_df

(8760, 44)


,Carbon monoxide,Nitrogen dioxide (NO2),Ozone,PM2.5 - Local Conditions,Wind Direction - Resultant,Wind Speed - Resultant,Outdoor Temperature,Relative Humidity,Solar radiation,Ultraviolet radiation,...,NH3,NO,OC,OTHER_AROM,OTHER_VOC,PRPE,SO2,TMB,TOLU,XYLE
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,1.4490,27.2,0.002,61.4,49.0,3.2,51.8,87.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 01:00:00,1.5300,27.8,0.001,NaN,35.0,2.9,51.4,84.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 02:00:00,1.4680,27.9,0.002,NaN,43.0,3.8,50.9,81.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 03:00:00,1.4590,28.7,0.001,NaN,38.0,3.9,50.3,81.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
2018-01-01 04:00:00,1.4395,27.9,0.002,NaN,36.0,4.2,49.5,79.0,0.0,0.0,...,1.385000e-11,3.272592e-10,9.679659e-12,1.605214e-11,3.551276e-11,1.917994e-12,8.529435e-11,1.532717e-12,5.400424e-11,5.694743e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,0.6230,32.4,0.007,NaN,32.0,1.8,55.4,15.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11
2018-12-31 20:00:00,0.8230,34.0,0.003,NaN,25.0,2.0,52.4,19.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11
2018-12-31 21:00:00,0.7425,31.7,0.005,NaN,30.0,2.3,51.7,18.0,0.0,0.0,...,1.391520e-11,3.342732e-10,9.898709e-12,1.605710e-11,3.551342e-11,1.957491e-12,8.600769e-11,1.533309e-12,5.402841e-11,5.695479e-11


## Compare CEDS Emissions data to PAMS_VOC Data

Which compounds appear in both datasets? What are the differences between the datasets.

It seeems that AQS reports verry granular data for speecific compounds, whereas CEDS providees mor generalized emissions data. For example, CEDS reports emissions for 'butanes' whereas AQS reports '2,2-dimethylbutane' instead. Furthermore, CEDS has more opaque naming conventions, like 'other alkanals' which might encompass any aldehydes in the AQS dataset. I am not sure how to match emissions to AQS VOC compounds without manually going through each compound and understanding the necessary chemistry.

For now: simply keep the ones that match in name exactly.

In [24]:
# NOTE: There are 60 PAMS_VOCS and LA North Main St has data on all of them:
print(f"Site has all PAMS_VOCS data: {len(voc_r['Metadata']['codes']) == len([x[1] for x in voc_r['Data']['Los Angeles-North Main Street'] if x[1] == True])}\n")

vocs = sorted([datafetcher.find_name(code) for code in voc_r['Metadata']['codes']])
print(f"AQS Compound Names:\n{vocs}\n")

print(f"CEDS Emissions Names:\n{datafetcher.ceds_compounds}\n")

Site has all PAMS_VOCS data: True

AQS Compound Names:
['1,2,3-Trimethylbenzene', '1,2,4-Trimethylbenzene', '1,3,5-Trimethylbenzene', '1-Butene', '1-Pentene', '2,2,4-Trimethylpentane', '2,2-Dimethylbutane', '2,3,4-Trimethylpentane', '2,3-Dimethylbutane', '2,3-Dimethylpentane', '2,4-Dimethylpentane', '2-Methylheptane', '2-Methylhexane', '2-Methylpentane', '3-Methylheptane', '3-Methylhexane', '3-Methylpentane', 'Acetaldehyde', 'Acetone', 'Acetylene', 'Benzene', 'Cyclohexane', 'Cyclopentane', 'Ethane', 'Ethylbenzene', 'Ethylene', 'Formaldehyde', 'Isobutane', 'Isopentane', 'Isoprene', 'Isopropylbenzene', 'Methylcyclohexane', 'Methylcyclopentane', 'Propane', 'Propylene', 'Styrene', 'Sum of PAMS target compounds', 'Toluene', 'Total NMOC (non-methane organic compound)', 'cis-2-Butene', 'cis-2-Pentene', 'm-Diethylbenzene', 'm-Ethyltoluene', 'm/p Xylene', 'n-Butane', 'n-Decane', 'n-Heptane', 'n-Hexane', 'n-Nonane', 'n-Octane', 'n-Pentane', 'n-Propylbenzene', 'n-Undecane', 'o-Ethyltoluene', 'o-X

## Match CEDS emissions data to AQS measurements

Using CEDS wiki page, match CEDS categories to the compounds in the AQS dataframe

In [45]:
# Compounds present in CEDS data
print(datafetcher.ceds_compounds.keys())

dict_keys(['ALD2', 'ALK4_butanes', 'ALK4_hexanes', 'ALK4_pentanes', 'BC', 'BENZ', 'BUTENE', 'C2H2', 'C2H4', 'C2H6', 'C3H8', 'CH2O', 'CHC', 'CO', 'CO2', 'EOH', 'ESTERS', 'ETHERS', 'HCOOH', 'MEK', 'N2O', 'NH3', 'NO', 'OC', 'OTHER_AROM', 'OTHER_VOC', 'PRPE', 'SO2', 'TMB', 'TOLU', 'XYLE'])


In [26]:
ceds_aqs_map = {
    'ALD2' : {'include' : True, 'matches' : ['Acetaldehyde'], 'notes':'not lumped'},
    'ALK4_butanes' : {'include' : True, 'matches' : ['2,2-Dimethylbutane', '2,3-Dimethylbutane', 'Isobutane', 'n-Butane', '2,2,3-Trimethylbutane'], 'notes' : 'lumped'}, 
    'ALK4_hexanes' : {'include' : True, 'matches' : ['n-Hexane','Cyclohexane', '3-Methylhexane','Methylcyclohexane','2-Methylhexane','2,4-Dimethylhexane','2,3-Dimethylhexane','3-Ethylhexane','Ethylcyclohexane','2-Methylhexane & cyclohexane','2,2,4-Trimethylhexane','2,5-Dimethylhexane'],  'notes' : 'lumped'},
    'ALK4_pentanes' : {'include' : True, 'matches' : ['n-Pentane','Isopentane','3-Methylpentane','Cyclopentane','2,2,4-Trimethylpentane','2,3,4-Trimethylpentane','Methylcyclopentane','2-Methylpentane','2,3-Dimethylpentane','2-2-3-Trimethylpentane','Isopentane & cyclopentane','Methylcyclopentane & 2,4-dimethylpentane','cis-1,2-Dimethylcyclopentane','trans-1,3-Dimethylcyclopentane', '2-Methylheptane', '2-Methylheptane & 3-Methylheptane', 'n-Heptane', '3-Methylheptane', 'Heptane','2,2-Dimethylheptane','4-Methylheptane', 'n-Octane', 'n-Nonane','Isomers of dodecane'], 'notes' : 'lumped'},
    'BC' : {'include' : True, 'matches' : ['Black carbon PM2.5 STP', 'Black Carbon PM2.5 Corrected', 'Black Carbon PM10 LC'], 'notes' : 'Black Carbon (aerosol)'},
    'BENZ' : {'include' : True, 'matches' : ['Benzene'], 'notes' : 'not lumped.'},
    'BUTENE' : {'include' : True, 'matches' : ['1-Butene', 'cis-2-Butene', 'trans-2-Butene', '1-Pentene','trans-2-Pentene,cis-2-Pentene','4-Methyl-1-pentene','2-Methyl-2-pentene','2,3-Dimethyl-2-pentene','2-Methyl-1-pentene','2,4,4-Trimethyl-1-pentene','2,4,4-Trimethyl-2-pentene','Cyclopentene','cis-4-Methyl-2-pentene','3-Methyl-1-butene & cyclopentene','2-Methyl-2-butene & 1-pentene','4-Methylpentene & 3-methylpentene','3-Methyl-1-butene & Cyclopentene','1-Hexene & 2-Methyl-1-pentene'], 'notes' : 'These stand for other_alkenes_and_alkynes <- super weird category, sometimes also called "isomers of pentene".'},
    'C2H2' : {'include' : True, 'matches' : ['Acetylene'], 'notes' : 'not lumped'},
    'C2H4' : {'include' : True, 'matches' : ['Ethylene'], 'notes' : 'not lumped'},
    'C2H6' : {'include' : True, 'matches' : ['Ethane'], 'notes' : 'not lumped'},
    'C3H8' : {'include' : True, 'matches' : ['Propane'], 'notes' : 'not lumped'},
    'CH2O' : {'include' : True, 'matches' : ['Formaldehyde'], 'notes' : 'not lumped'},
    'CHC' : {'include' : True, 'matches' : [], 'notes' : 'lumped. There are a bunch, but we dn not need them. Some names are “obvious” like 1,1,1-Trichloro-2,2-bis (p-chlorophenyl) ethane but others are pesticides with chemical company names like Endrin'},
    'CO' : {'include' : True, 'matches' : ['CO'],'notes' : ''},
    'CO2' : {'include' : False, 'matches' : [], 'notes' : ''},
    'EOH' : {'include' : False, 'matches' : [], 'notes' : 'not lumped (ethanol)'},
    'ESTERS' : {'include' : True, 'matches' : ['n-Propyl acetate', 'Ethyl acrylate', 'n-Butyl acrylate', 'Methyl methacrylate', 'Vinyl acetate', 'Butyl acetate'], 'notes' : 'lumped'},
    'ETHERS' : {'include' : False, 'matches' : [], 'notes' :'probably can just ignore these'},
    'HCOOH' : {'include' : False, 'matches' : [], 'notes' : 'Does not look like it is in the EPA dataset'},
    'MEK' : {'include' : True, 'matches' : ['Methyl ethyl ketone & methacrolein', 'Methyl ethyl ketone', 'Methacrolein'], 'notes' : 'MEK and Methacrolein are usually lumped together - same mass so measured as one often'},
    'N2O' : {'include' : False, 'matches' : [], 'notes' : 'It is code 42605 - we can skip it'},
    'NH3' : {'include' : True, 'matches' : ['Ammonia'], 'notes' : 'only 42604.'},
    'NO' : {'include' : True, 'matches' : ['Nitric oxide (NO)'], 'notes' : 'I am not sure if any of these are nitric oxide.'},
    'OC' : {'include' : True, 'matches' : ['Organic carbon PM10 STP', 'Organic Carbon Mass PM2.5 LC'], 'notes' : 'Organic Carbon, meaning organic aerosol.'},
    'OTHER_AROM' : {'include' : False, 'matches' : [], 'notes': 'pretty much everything else that ends with “benzene”, “toluene”,or “xylene” that is not included in BENZ, TOLU, XYLE, and TMB goes here. We can skip them though'},
    'OTHER_VOC' : {'include' : False, 'matches' : [], 'notes' : 'many of the AQS ones could, but anything relegated to “other” takes part in almost no interesting chemistry, so we can skip them'},
    'PRPE' : {'include' : True, 'matches' : ['Propylene'], 'notes' : ''},
    'SO2' : {'include' : True, 'matches' : ['Sulfur dioxide', 'SO2 max 5-min avg'], 'notes' : ''},
    'TMB' : {'include' : True, 'matches' : ['1,2,3-Trimethylbenzene', '1,2,4-Trimethylbenzene', '1,3,5-Trimethylbenzene'], 'notes' : ''},
    'TOLU' : {'include' : True, 'matches' : ['Toluene'], 'notes' : 'not lumped'},
    'XYLE' : {'include' : True, 'matches' : ['m/p Xylene','m-Xylene', 'o-Xylene','p-Xylene','Xylene(s)'], 'notes' : ''}
}

In [40]:
# Get all vocs in AQS that have emissions recorded by CEDS and
matched_vocs = set()
for k in ceds_aqs_map:
    for match in ceds_aqs_map[k]['matches']:
        matched_vocs.add(match)
unmatched_vocs = [x for x in vocs if x not in matched_vocs]
unmatched_vocs

['2,4-Dimethylpentane',
 'Acetone',
 'Ethylbenzene',
 'Isoprene',
 'Isopropylbenzene',
 'Styrene',
 'Sum of PAMS target compounds',
 'Total NMOC (non-methane organic compound)',
 'cis-2-Pentene',
 'm-Diethylbenzene',
 'm-Ethyltoluene',
 'n-Decane',
 'n-Propylbenzene',
 'n-Undecane',
 'o-Ethyltoluene',
 'p-Diethylbenzene',
 'p-Ethyltoluene',
 'trans-2-Pentene']

### NOTES ON UNMATCHED AQS COMPOUNDS

* Do we drop the Sum of PAMS target compounds?
* What are non-methane organic compounds? Is that anything without a methane group?
* Are all the remaining compounds just under a broader category in CEDS that I missed?

In [48]:
print(f"Matched {len(matched_vocs)} compounds.")
# Get only compounds that are in the vocs list, and the matched list
final_vocs = [x for x in vocs if x in matched_vocs]
print(f"Using {len(final_vocs)} AQS VOCs.")
# Get all emissions recorded by CEDS that are used by a VOC in AQS data
final_emissions = [k for k in ceds_aqs_map if len(set(ceds_aqs_map[k]['matches']) & set(final_vocs)) != 0]
print(f"Using emissions data for {len(final_emissions)} compounds.")

Matched 95 compounds.
Using 41 AQS VOCs.
Using emissions data for 15 compounds.


### Join CRITERIA, MET, PAMS_VOCS, and Emissions data into one df

Jupyter notebooks are really slow and memory inefficient so we choose to do this in a python script. I think this is the end of this notebook and I am going to create a new notebook that starts out with a dataset and explores how we can apply models to it.

In [47]:
vocs_data = datafetcher.get_voc_data(20180101, 20181231, '06', '037', '1103', matched_vocs)
emissions_data = datafetcher.get_ceds_data() # Default parameters for LA North Main St 2018

No data for 2,2,3-Trimethylbutane
No data for Ethylcyclohexane
No data for Cyclopentene
No data for Organic carbon PM10 STP
Could not find trans-2-Pentene,cis-2-Pentene.
{'Header': [{'status': 'Failed', 'request_time': '2022-04-07T00:50:08.664-04:00', 'url': 'https://aqs.epa.gov/data/api/sampleData/bySite?email=tcintra%40hmc.edu&key=russetwren13&bdate=20180101&edate=20181231&state=06&county=037&site=1103', 'error': ['variable is missing or the value is empty: param']}]}


AttributeError: 'NoneType' object has no attribute 'empty'